<h1><center>Laboratorio 8: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz.

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Nicolás Gatica    
- Nombre de alumno 2: Cristóbal Igor


### **Link de repositorio de GitHub:** `https://github.com/ngatica/LaboratoriosMDS`

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- Fecha de entrega: 23/06/2022
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [42]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/cvis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [3]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [4]:
df_comics = pd.read_csv('df_comics.csv')
df_comics_no_label = pd.read_csv('comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [5]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/cvis/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [8]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('english')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to /home/cvis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['rock', 'best', 'actor', 'world'],
 ['new', 'york', 'beauti', 'citi']]

In [9]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [10]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,actor,beauti,best,citi,good,like,new,rock,roll,teacher,world,york
0,1,0,0,0,0,1,1,0,2,1,1,0,0
1,0,1,0,1,0,0,0,0,1,0,0,1,0
2,0,0,1,0,1,0,0,1,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [11]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,& roll,actor,actor world,beauti,beauti citi,best,best actor,citi,good,...,rock,rock &,rock best,rock like,roll,teacher,teacher rock,world,york,york beauti
0,1,1,0,0,0,0,0,0,0,1,...,2,1,0,1,1,1,1,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,1,0,1,0,0,0,0,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [19]:
#### Código aquí ####
bog = CountVectorizer(tokenizer= StemmerTokenizer(),
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']

preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('bog',  bog, 'history_text'),
        ('MinMaxScaler', MinMaxScaler(), atributos_de_interes)])                      

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [ ] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [ ] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

In [51]:
#### Código aquí ####
# Se crea el pipeline con el prepocesamiento seteado en la sección anterior, el mejor percentil seleccionado y el clasificador mulitinomialNB
pipe = Pipeline(steps=[("preprocessing", preprocessing_transformer),
                        ("bf", SelectPercentile(f_classif, percentile=90)),
                        ("clf", MultinomialNB())])

y = df_comics["alignment"]
X = df_comics.drop(columns = ["alignment"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

In [52]:
report = classification_report(y_test, y_pred, zero_division=0)
print(report)

              precision    recall  f1-score   support

         Bad       0.69      0.10      0.18        86
        Good       0.59      0.98      0.74       148
     Neutral       0.00      0.00      0.00        23

    accuracy                           0.60       257
   macro avg       0.43      0.36      0.31       257
weighted avg       0.57      0.60      0.49       257



```
Comentar los resultados
```

Respuesta: 
Dentro de este primer modelo clasificador se puede comentar que a primera vista los resultados son bastantes malos sobre todo en la clasificación de las clases minoritarias, esto debido a que el modelo se sobreajustó en la clase "Good" que es la que presenta el mejor recall. El problema con las clases minoritarias se refleja principalmente en los bajos recall, lo cual significa que no está identificando los casos reales de esa clase (sobre todo en Bad donde la precision es 0.69 pero con un bajísimo recall)

```
¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?
```
Respuesta:
No es recomendable utilizar este clasificador para predecir si BatCow es Bueno, Malo o Neutral ya que el modelo se encuentra sobre entrenado para la clase Bueno.



In [53]:
from sklearn.dummy import DummyClassifier
# Pipeline con el clasificador dummy
dummy_pipe = Pipeline(steps=[("preprocessing", preprocessing_transformer),
                        ("bf", SelectPercentile(f_classif, percentile=90)),
                        ("clf", DummyClassifier(strategy="stratified", random_state=42))])

dummy_pipe.fit(X_train, y_train)
dummy_y_pred = dummy_pipe.predict(X_test)

In [54]:
dummy_report = classification_report(y_test, dummy_y_pred)
print(dummy_report)

              precision    recall  f1-score   support

         Bad       0.32      0.34      0.33        86
        Good       0.58      0.56      0.57       148
     Neutral       0.12      0.13      0.12        23

    accuracy                           0.45       257
   macro avg       0.34      0.34      0.34       257
weighted avg       0.46      0.45      0.45       257



```
Comentar que implican los scores obtenidos en comparación con los resultados del baseline
```

Respuesta:
El clasificador Dummy, configurado con la estrategia de tipo "Stratified", no toma en consideración los atributos de los datos sino que construye distribuciones multinomiales parametrizadas a partir de las probabilidades prior de las clases. Esto es jugar mucho con las probabilidad y dado que no son muchos ejemplos esto no se prodría considera representativo y así también hace referencia su nombre "dummy".

Ahora, contrastando este clasificador dummy con el baseline obtenido en la celda anterior, se tiene que los resultados no se encuentran muy alejados y tomando en consideración que el dummy es el clasificador más básico a realizar el baseline se encuentra en una escala de resultados sumamente pobre.

```
¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?
```
Respuesta:
En base a lo comentado anteriormente, el clasificador entrenado no mejora los resultados del dummy

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [28]:
pipe.get_params()

{'memory': None,
 'steps': [('preprocessing',
   ColumnTransformer(transformers=[('bog',
                                    CountVectorizer(ngram_range=(1, 2),
                                                    tokenizer=<__main__.StemmerTokenizer object at 0x7fbf822fcbd0>),
                                    'history_text'),
                                   ('MinMaxScaler', MinMaxScaler(),
                                    ['intelligence_score', 'strength_score',
                                     'speed_score', 'durability_score',
                                     'power_score', 'combat_score'])])),
  ('bf', SelectPercentile(percentile=90)),
  ('clf', MultinomialNB())],
 'verbose': False,
 'preprocessing': ColumnTransformer(transformers=[('bog',
                                  CountVectorizer(ngram_range=(1, 2),
                                                  tokenizer=<__main__.StemmerTokenizer object at 0x7fbf822fcbd0>),
                                  'history_te

In [43]:
param_grid = [
    # clf1
    {'clf': [MultinomialNB()],
    "bf__percentile": [20, 40, 60, 80],
    "preprocessing__bog__ngram_range": [(1, 1), (1, 2), (1, 3)],
    'preprocessing__bog__max_df': [0.6, 1.0],
    'preprocessing__bog__min_df': [0.1, 0.3],
    },
    
    # clf2
    {'clf': [ComplementNB()],
    "bf__percentile": [20, 40, 60, 80],
    "preprocessing__bog__ngram_range": [(1, 1), (1, 2), (1, 3)],
    'preprocessing__bog__max_df': [0.6, 1.0],
    'preprocessing__bog__min_df': [0.1, 0.3]
    },
    
    # clf3
    {'clf': [GradientBoostingClassifier(random_state=666)],
    "bf__percentile": [20, 40, 60, 80],
    "preprocessing__bog__ngram_range": [(1, 1), (1, 2), (1, 3)],
    'preprocessing__bog__max_df': [0.6, 1.0],
    'preprocessing__bog__min_df': [0.1, 0.3],
    'clf__max_depth': [3,5,8],
    'clf__n_estimators': [200, 500],
    'clf__max_features': ['log2','sqrt'],
    'clf__criterion': ['friedman_mse',  'absolute_error']
    }    
]

param_grid

[{'clf': [MultinomialNB()],
  'bf__percentile': [20, 40, 60, 80],
  'preprocessing__bog__ngram_range': [(1, 1), (1, 2), (1, 3)],
  'preprocessing__bog__max_df': [0.6, 1.0],
  'preprocessing__bog__min_df': [0.1, 0.3]},
 {'clf': [ComplementNB()],
  'bf__percentile': [20, 40, 60, 80],
  'preprocessing__bog__ngram_range': [(1, 1), (1, 2), (1, 3)],
  'preprocessing__bog__max_df': [0.6, 1.0],
  'preprocessing__bog__min_df': [0.1, 0.3]},
 {'clf': [GradientBoostingClassifier(random_state=666)],
  'bf__percentile': [20, 40, 60, 80],
  'preprocessing__bog__ngram_range': [(1, 1), (1, 2), (1, 3)],
  'preprocessing__bog__max_df': [0.6, 1.0],
  'preprocessing__bog__min_df': [0.1, 0.3],
  'clf__max_depth': [3, 5, 8],
  'clf__n_estimators': [200, 500],
  'clf__max_features': ['log2', 'sqrt'],
  'clf__criterion': ['friedman_mse', 'absolute_error']}]

In [55]:
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV
gs = HalvingGridSearchCV(pipe, param_grid, n_jobs=-1, scoring="balanced_accuracy", verbose=10, random_state=33)
gs.fit(X_train, y_train)

n_iterations: 4
n_required_iterations: 7
n_possible_iterations: 4
min_resources_: 30
max_resources_: 1028
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 1248
n_resources: 30
Fitting 5 folds for each of 1248 candidates, totalling 6240 fits
----------
iter: 1
n_candidates: 416
n_resources: 90
Fitting 5 folds for each of 416 candidates, totalling 2080 fits
----------
iter: 2
n_candidates: 139
n_resources: 270
Fitting 5 folds for each of 139 candidates, totalling 695 fits
----------
iter: 3
n_candidates: 47
n_resources: 810
Fitting 5 folds for each of 47 candidates, totalling 235 fits


HalvingGridSearchCV(estimator=Pipeline(steps=[('preprocessing',
                                               ColumnTransformer(transformers=[('bog',
                                                                                CountVectorizer(ngram_range=(1,
                                                                                                             2),
                                                                                                tokenizer=<__main__.StemmerTokenizer object at 0x7fbf822fcbd0>),
                                                                                'history_text'),
                                                                               ('MinMaxScaler',
                                                                                MinMaxScaler(),
                                                                                ['intelligence_score',
                                                                       

In [56]:
gs.best_score_

0.4692624896293318

In [57]:
gs.best_estimator_

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('bog',
                                                  CountVectorizer(max_df=0.6,
                                                                  min_df=0.1,
                                                                  ngram_range=(1,
                                                                               3),
                                                                  tokenizer=<__main__.StemmerTokenizer object at 0x7fbf87192090>),
                                                  'history_text'),
                                                 ('MinMaxScaler',
                                                  MinMaxScaler(),
                                                  ['intelligence_score',
                                                   'strength_score',
                                                   'speed_score',
                                                  

In [77]:
new_pipe = gs.best_estimator_

In [60]:
a_report = classification_report(y_test, gs.best_estimator_.predict(X_test))
print(a_report)

              precision    recall  f1-score   support

         Bad       0.46      0.69      0.55        86
        Good       0.80      0.56      0.66       148
     Neutral       0.12      0.13      0.12        23

    accuracy                           0.56       257
   macro avg       0.46      0.46      0.45       257
weighted avg       0.63      0.56      0.58       257



```
Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.
```

La mejor combinación encontrada para el modelo según la grilla resultó ser el clasificador MultinomialNB, esto dado que es el mejor modelo para trabajar con texto ya que las distribuciones de las palabras son importantes (además Scikit recomienda para problemas de clasificación de texto utilizar modelos basados en Naive Bayes https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html). En cuanto a las features utilizadas sólo se utilizó las que abarcaban el 80%, por lo que un 20% de las columnas no son relevantes a este problema. Por otra parte, el modelo arroja utilizar la mayor cantidad de n-gramas, lo cual tiene sentido dado que es importante tener el mayor contexto posible al vectorizar. 
Por último, en los mejores parámetros del CountVectorizer se obtuvo mejores resultados con las combinaciones de min_df=0.1 y max_df=0.6, lo cual significa que palabras muy repetidas no son consideradas (sobre el 60%) y pocas repetidas tampoco (bajo el 10%).

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [61]:
#### Código aquí ####
df_comics_no_label

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,1,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,NaN,100,20,30,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,14,Ace Morgan,Kyle Morgan,NaN,7,NaN,"Aviation: Ace is an extremely skilled pilot, ...",85,20,65,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
2,17,A'dal,NaN,NaN,7,"As with most of the naaru, little is known of ...",NaN,85,30,70,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27,Agent Zero (FOX),David North,NaN,6,"During mid-late 1973, Zero was a member of Tea...",Zero can absorb kinetic energy to further incr...,90,10,25,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,31,Ajax (FOX),Francis,Francis Freeman,7,Ajax (born Francis Freeman) was a human who ga...,Ajax has claimed that the procedure to obtain ...,85,25,45,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,1394,Weapon Hex,Laura Kinney,NaN,-,NaN,NONE NOTABLE,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,1396,Wendigo,Wendigo,NaN,6,The Wendigo are the result of an ancient curse...,NaN,65,80,75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
81,1431,XS (CW),Nora West-Allen,NaN,19,"According to Eobard Thawne, the daughter of Ba...",Nora possesses a natural connection to the Spe...,90,55,100,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
82,1440,Yondu (MCU),Yondu,NaN,7,"Yondu Udonta was a Centaurian, leader of a key...","Yondu Udonta was a member of the Centaurians, ...",85,25,40,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
df_comics_no_label = df_comics_no_label.dropna(subset = ["history_text"]) #Eliminamos todos las nan del de history_test

In [63]:
# Predecimos
best_pipe = gs.best_estimator_
#best_pipe.fit(X_train, y_train) #Entrenamiento
y_pred = best_pipe.predict(df_comics_no_label.drop(columns=["alignment"]))

In [64]:
df_comics_no_label['pred'] = y_pred
nombres = ['Vergil', 'Gorilla Girl', 'Batcow']
df_mask = df_comics_no_label[df_comics_no_label.name.isin(nombres)]
df_mask[['name', 'real_name', 'full_name', 'overall_score', 'history_text', 'powers_text', 'pred']]


/home/cvis/miniconda3/envs/universidad/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,name,real_name,full_name,overall_score,history_text,powers_text,pred
16,Batcow,None,None,3,Bat-Cow was originally a cow that was found by...,NaN,Good
40,Gorilla Girl,Fahnbullah Eddy,Fahnbullah Eddy,7,A carnival performer with the ability to turn ...,Gorilla Girl can transform into a talking gori...,Good
78,Vergil,Vergil Sparda,NaN,16,"Vergil, later also known as Nelo Angelo, is on...",NaN,Bad
83,Batcow,None,None,3,Bat-Cow was originally a cow that was found by...,NaN,Good


In [76]:
df_mask.history_text.values

array(['Bat-Cow was originally a cow that was found by Robin during an investigation on a slaughterhouse, which the cow was sent to. Robin wanted to keep the cow and named it Bat-Cow. Batman decided they would keep the cow. During a later investigation, Batman discovered it had mind-controlling radiation inside of it, leading him to attempt to get rid of the cow. Robin, however would not allow him to hurt the cow. After the events of Robin\'s death, Bat-Cow and Robin\'s dog felt a great loss in morale. Alfred Pennyworth and Nightwing had to take care of the cow. Later after Nightwing\'s disappearance, Alfred was the only caretaker of Bat-Cow. Bat-Cow, complete with cape, spent time on a Wayne Enterprises-owned dairy farm. It was there the cow encountered the cosmic-powered "Forever People." One of them seemed to gain guidance from Bat-Cow\'s actions, namely when Bat-Cow gave one of the Forever People a scarecrow\'s hat. Bat-Cow formed a friendship with Titus, a dog and household cat.',

Dentro de los resultados se puede ver que Batcow es clasificade como Good, Gorilla Girl como Good y por último Vergil como un personaje malo, lo cual calza con lo de las historias.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>